In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

Introduction to `torch.compile`
===============================

**Author:** William Wen


`torch.compile` is the latest method to speed up your PyTorch code!
`torch.compile` makes PyTorch code run faster by JIT-compiling PyTorch
code into optimized kernels, all while requiring minimal code changes.

In this tutorial, we cover basic `torch.compile` usage, and demonstrate
the advantages of `torch.compile` over previous PyTorch compiler
solutions, such as
[TorchScript](https://pytorch.org/docs/stable/jit.html) and [FX
Tracing](https://pytorch.org/docs/stable/fx.html#torch.fx.symbolic_trace).

**Contents**

::: {.contents local=""}
:::

**Required pip Dependencies**

-   `torch >= 2.0`
-   `torchvision`
-   `numpy`
-   `scipy`
-   `tabulate`


NOTE: a modern NVIDIA GPU (H100, A100, or V100) is recommended for this
tutorial in order to reproduce the speedup numbers shown below and
documented elsewhere.


In [1]:
import torch
import warnings

gpu_ok = False
if torch.cuda.is_available():
    device_cap = torch.cuda.get_device_capability()
    if device_cap in ((7, 0), (8, 0), (9, 0)):
        gpu_ok = True

if not gpu_ok:
    warnings.warn(
        "GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower "
        "than expected."
    )

/tmp/ipykernel_2246949/231343347.py:11: UserWarning: GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
  warnings.warn(


Basic Usage
===========

`torch.compile` is included in the latest PyTorch. Running TorchInductor
on GPU requires Triton, which is included with the PyTorch 2.0 nightly
binary. If Triton is still missing, try installing `torchtriton` via pip
(`pip install torchtriton --extra-index-url "https://download.pytorch.org/whl/nightly/cu117"`
for CUDA 11.7).

Arbitrary Python functions can be optimized by passing the callable to
`torch.compile`. We can then call the returned optimized function in
place of the original function.


In [2]:
def foo(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b
opt_foo1 = torch.compile(foo)
print(opt_foo1(torch.randn(10, 10), torch.randn(10, 10)))

tensor([[ 1.5335,  0.3465,  1.9143,  0.9716, -0.0429,  1.9355,  1.0676,  0.3306,
          1.3733,  1.4190],
        [ 1.1277,  1.7637, -0.2343, -0.6571,  0.4945, -0.0883, -0.4642,  0.6578,
          0.4976,  1.3901],
        [ 1.2774,  0.4842,  0.4335,  0.9112,  1.7328,  0.6462,  0.1407,  0.8807,
          0.9457,  0.3807],
        [ 0.8419, -0.8132,  1.8676,  0.8064,  1.3425,  1.8061,  1.6608,  1.0079,
         -0.1747,  0.4780],
        [-0.8330,  1.2719, -0.5550,  1.1353,  0.8251,  0.1587,  1.6969,  1.0465,
          0.7149, -0.2530],
        [ 0.6498,  1.4223,  1.1186,  1.3495,  1.0687,  0.2908, -0.3300,  1.3293,
          0.5939,  0.1200],
        [ 1.4036, -0.5386,  0.0090,  1.2330,  1.0574, -0.2328,  0.3748,  0.2190,
          0.3048,  1.7923],
        [ 1.9219,  1.4021,  1.7602,  0.6483,  1.8685, -0.9550, -0.0489,  1.6960,
          0.7117, -0.1962],
        [-0.0614, -0.0236,  0.2132,  0.5960, -0.0352,  0.7457,  0.3145,  0.5131,
          0.3553,  0.0564],
        [-0.1826,  

Alternatively, we can decorate the function.


In [3]:
@torch.compile
def opt_foo2(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b
print(opt_foo2(torch.randn(10, 10), torch.randn(10, 10)))

tensor([[ 0.8802,  0.7461,  0.0658,  1.8170,  0.9049, -0.5295, -0.2082,  1.8852,
          1.1846, -0.2176],
        [ 1.1182,  1.6383,  1.0807,  0.1714,  0.2397,  1.0027,  0.5430,  0.1654,
          1.8210,  1.2273],
        [ 0.1788, -1.2831,  1.5140,  0.9588,  1.0260,  0.6011,  0.2790,  0.0081,
          0.5861,  0.1106],
        [ 0.1722,  0.3267,  0.1772,  0.5518,  0.0229, -0.6844, -0.0426,  0.1556,
          1.7909,  0.3525],
        [-0.0908,  0.5472, -0.4691,  1.6457, -0.0072,  1.5979,  1.5622, -0.5661,
          1.8068,  0.4858],
        [ 1.8913,  0.3078,  1.2601,  0.4763,  1.4484,  0.6009, -1.4493,  1.4023,
         -0.1619,  0.3959],
        [ 1.6603,  1.4932,  0.3248,  1.0420,  0.0293,  0.4254,  0.1202,  0.0654,
          0.4750,  1.6710],
        [ 1.1983,  1.3269,  0.6009,  0.0827,  1.6679,  1.0140,  1.4927,  0.7823,
          0.8853, -0.0286],
        [-0.9299,  0.5703,  1.1296,  0.6993,  1.8115,  1.5370,  1.8766,  1.5094,
          1.6158,  1.8214],
        [ 1.9047,  

We can also optimize `torch.nn.Module` instances.


In [4]:
class MyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = torch.nn.Linear(100, 10)

    def forward(self, x):
        return torch.nn.functional.relu(self.lin(x))

mod = MyModule()
opt_mod = torch.compile(mod)
print(opt_mod(torch.randn(10, 100)))

tensor([[0.7640, 0.0000, 0.7013, 0.0000, 0.2067, 0.3552, 0.0000, 0.0000, 0.2215,
         0.0000],
        [0.0000, 0.0000, 0.4442, 0.0000, 0.0000, 0.7740, 0.0000, 0.0000, 0.6380,
         0.0000],
        [0.3507, 0.0170, 0.3435, 0.7555, 0.0000, 0.0000, 0.8052, 0.8096, 0.0000,
         0.0000],
        [0.0000, 1.0629, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4021, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.4148, 0.0000, 0.0000, 0.1305, 0.0000, 0.4312,
         0.8549],
        [0.5396, 0.0000, 0.8912, 0.0000, 0.0000, 0.4437, 0.0000, 0.4966, 0.1126,
         0.0000],
        [0.1113, 0.2187, 0.0579, 0.0000, 0.5500, 0.2027, 0.1270, 0.6782, 0.0000,
         0.1334],
        [0.0000, 0.9013, 0.2160, 0.6244, 0.0000, 0.0000, 0.1591, 0.5306, 0.3828,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1430, 0.0000, 0.0000, 0.4810,
         0.0000],
        [0.7683, 0.0000, 0.2170, 0.3186, 0.2797, 0.6242, 0.3370, 0.2197, 0.2087,
         -0.0000]], grad_fn=

Demonstrating Speedups
======================

Let\'s now demonstrate that using `torch.compile` can speed up real
models. We will compare standard eager mode and `torch.compile` by
evaluating and training a `torchvision` model on random data.

Before we start, we need to define some utility functions.


In [5]:
# Returns the result of running `fn()` and the time it took for `fn()` to run,
# in seconds. We use CUDA events and synchronization for the most accurate
# measurements.
def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000

# Generates random input and targets data for the model, where `b` is
# batch size.
def generate_data(b):
    return (
        torch.randn(b, 3, 128, 128).to(torch.float32).cuda(),
        torch.randint(1000, (b,)).cuda(),
    )

N_ITERS = 10

from torchvision.models import densenet121
def init_model():
    return densenet121().to(torch.float32).cuda()

First, let\'s compare inference.

Note that in the call to `torch.compile`, we have have the additional
`mode` argument, which we will discuss below.


In [6]:
model = init_model()

# Reset since we are using a different mode.
import torch._dynamo
torch._dynamo.reset()

model_opt = torch.compile(model, mode="reduce-overhead")

inp = generate_data(16)[0]
with torch.no_grad():
    print("eager:", timed(lambda: model(inp))[1])
    print("compile:", timed(lambda: model_opt(inp))[1])

eager: 10.801107421875


/data/ywang/anaconda3/envs/diffusion/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:90: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
[2024-03-15 11:42:36,165] torch._inductor.utils: [WARNING] skipping cudagraphs due to input mutation


compile: 31.843275390625


Notice that `torch.compile` takes a lot longer to complete compared to
eager. This is because `torch.compile` compiles the model into optimized
kernels as it executes. In our example, the structure of the model
doesn\'t change, and so recompilation is not needed. So if we run our
optimized model several more times, we should see a significant
improvement compared to eager.


In [7]:
eager_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    with torch.no_grad():
        _, eager_time = timed(lambda: model(inp))
    eager_times.append(eager_time)
    print(f"eager eval time {i}: {eager_time}")

print("~" * 10)

compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    with torch.no_grad():
        _, compile_time = timed(lambda: model_opt(inp))
    compile_times.append(compile_time)
    print(f"compile eval time {i}: {compile_time}")
print("~" * 10)

import numpy as np
eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
assert(speedup > 1)
print(f"(eval) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager eval time 0: 0.02570841598510742
eager eval time 1: 0.02353561592102051
eager eval time 2: 0.022993984222412108
eager eval time 3: 0.02270128059387207
eager eval time 4: 0.02248512077331543
eager eval time 5: 0.02248806381225586
eager eval time 6: 0.022800416946411134
eager eval time 7: 0.023765920639038086
eager eval time 8: 0.02255462455749512
eager eval time 9: 0.022867168426513672
~~~~~~~~~~
compile eval time 0: 0.021899200439453124
compile eval time 1: 0.017100799560546876
compile eval time 2: 0.016371551513671874
compile eval time 3: 0.01601456069946289
compile eval time 4: 0.016035039901733397
compile eval time 5: 0.01620172882080078
compile eval time 6: 0.016526336669921874
compile eval time 7: 0.016264192581176756
compile eval time 8: 0.01585855960845947
compile eval time 9: 0.015949024200439452
~~~~~~~~~~
(eval) eager median: 0.022833792686462403, compile median: 0.016232960700988768, speedup: 1.4066314276896863x
~~~~~~~~~~


And indeed, we can see that running our model with `torch.compile`
results in a significant speedup. Speedup mainly comes from reducing
Python overhead and GPU read/writes, and so the observed speedup may
vary on factors such as model architecture and batch size. For example,
if a model\'s architecture is simple and the amount of data is large,
then the bottleneck would be GPU compute and the observed speedup may be
less significant.

You may also see different speedup results depending on the chosen
`mode` argument. The `"reduce-overhead"` mode uses CUDA graphs to
further reduce the overhead of Python. For your own models, you may need
to experiment with different modes to maximize speedup. You can read
more about modes
[here](https://pytorch.org/get-started/pytorch-2.0/#user-experience).

You may might also notice that the second time we run our model with
`torch.compile` is significantly slower than the other runs, although it
is much faster than the first run. This is because the
`"reduce-overhead"` mode runs a few warm-up iterations for CUDA graphs.

For general PyTorch benchmarking, you can try using
`torch.utils.benchmark` instead of the `timed` function we defined
above. We wrote our own timing function in this tutorial to show
`torch.compile`\'s compilation latency.

Now, let\'s consider comparing training.


In [8]:
model = init_model()
opt = torch.optim.Adam(model.parameters())

def train(mod, data):
    opt.zero_grad(True)
    pred = mod(data[0])
    loss = torch.nn.CrossEntropyLoss()(pred, data[1])
    loss.backward()
    opt.step()

eager_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, eager_time = timed(lambda: train(model, inp))
    eager_times.append(eager_time)
    print(f"eager train time {i}: {eager_time}")
print("~" * 10)

model = init_model()
opt = torch.optim.Adam(model.parameters())
train_opt = torch.compile(train, mode="reduce-overhead")

compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, compile_time = timed(lambda: train_opt(model, inp))
    compile_times.append(compile_time)
    print(f"compile train time {i}: {compile_time}")
print("~" * 10)

eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
assert(speedup > 1)
print(f"(train) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager train time 0: 2.522144775390625
eager train time 1: 0.073657470703125
eager train time 2: 0.06748057556152344
eager train time 3: 0.07328665924072265
eager train time 4: 0.07324467468261718
eager train time 5: 0.06745088195800782
eager train time 6: 0.06795692443847656
eager train time 7: 0.06980095672607421
eager train time 8: 0.0676923828125
eager train time 9: 0.06778368377685547
~~~~~~~~~~
compile train time 0: 67.0295859375
compile train time 1: 0.033732608795166014
compile train time 2: 0.026844160079956055
compile train time 3: 0.027304960250854493
compile train time 4: 0.027595775604248047
compile train time 5: 0.028115968704223632
compile train time 6: 0.02954547119140625
compile train time 7: 0.03171225547790527
compile train time 8: 0.03427020645141601
compile train time 9: 0.03310899353027344
~~~~~~~~~~
(train) eager median: 0.0688789405822754, compile median: 0.03062886333465576, speedup: 2.248824575358651x
~~~~~~~~~~


Again, we can see that `torch.compile` takes longer in the first
iteration, as it must compile the model, but in subsequent iterations,
we see significant speedups compared to eager.

We remark that the speedup numbers presented in this tutorial are for
demonstration purposes only. Official speedup values can be seen at the
[TorchInductor performance
dashboard](https://hud.pytorch.org/benchmark/compilers).


Comparison to TorchScript and FX Tracing
========================================

We have seen that `torch.compile` can speed up PyTorch code. Why else
should we use `torch.compile` over existing PyTorch compiler solutions,
such as TorchScript or FX Tracing? Primarily, the advantage of
`torch.compile` lies in its ability to handle arbitrary Python code with
minimal changes to existing code.

One case that `torch.compile` can handle that other compiler solutions
struggle with is data-dependent control flow (the `if x.sum() < 0:` line
below).


In [9]:
def f1(x, y):
    if x.sum() < 0:
        return -y
    return y

# Test that `fn1` and `fn2` return the same result, given
# the same arguments `args`. Typically, `fn1` will be an eager function
# while `fn2` will be a compiled function (torch.compile, TorchScript, or FX graph).
def test_fns(fn1, fn2, args):
    out1 = fn1(*args)
    out2 = fn2(*args)
    return torch.allclose(out1, out2)

inp1 = torch.randn(5, 5)
inp2 = torch.randn(5, 5)

TorchScript tracing `f1` results in silently incorrect results, since
only the actual control flow path is traced.


In [10]:
traced_f1 = torch.jit.trace(f1, (inp1, inp2))
print("traced 1, 1:", test_fns(f1, traced_f1, (inp1, inp2)))
print("traced 1, 2:", test_fns(f1, traced_f1, (-inp1, inp2)))

/tmp/ipykernel_2246949/4005623017.py:2: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() < 0:


traced 1, 1: True
traced 1, 2: False


FX tracing `f1` results in an error due to the presence of
data-dependent control flow.


In [11]:
import traceback as tb
try:
    torch.fx.symbolic_trace(f1)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_2246949/1451191637.py", line 3, in <module>
    torch.fx.symbolic_trace(f1)
  File "/data/ywang/anaconda3/envs/diffusion/lib/python3.10/site-packages/torch/fx/_symbolic_trace.py", line 1109, in symbolic_trace
    graph = tracer.trace(root, concrete_args)
  File "/data/ywang/anaconda3/envs/diffusion/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py", line 209, in _fn
    return fn(*args, **kwargs)
  File "/data/ywang/anaconda3/envs/diffusion/lib/python3.10/site-packages/torch/fx/_symbolic_trace.py", line 778, in trace
    (self.create_arg(fn(*args)),),
  File "/tmp/ipykernel_2246949/4005623017.py", line 2, in f1
    if x.sum() < 0:
  File "/data/ywang/anaconda3/envs/diffusion/lib/python3.10/site-packages/torch/fx/proxy.py", line 413, in __bool__
    return self.tracer.to_bool(self)
  File "/data/ywang/anaconda3/envs/diffusion/lib/python3.10/site-packages/torch/fx/proxy.py", line 276, in to_bool
    raise TraceError('symbol

If we provide a value for `x` as we try to FX trace `f1`, then we run
into the same problem as TorchScript tracing, as the data-dependent
control flow is removed in the traced function.


In [12]:
fx_f1 = torch.fx.symbolic_trace(f1, concrete_args={"x": inp1})
print("fx 1, 1:", test_fns(f1, fx_f1, (inp1, inp2)))
print("fx 1, 2:", test_fns(f1, fx_f1, (-inp1, inp2)))

fx 1, 1: True
fx 1, 2: False


/data/ywang/anaconda3/envs/diffusion/lib/python3.10/site-packages/torch/fx/_symbolic_trace.py:602: UserWarning: Was not able to add assertion to guarantee correct input x to specialized function. It is up to the user to make sure that your inputs match the inputs you specialized the function with.
  warnings.warn(


Now we can see that `torch.compile` correctly handles data-dependent
control flow.


In [13]:
# Reset since we are using a different mode.
torch._dynamo.reset()

compile_f1 = torch.compile(f1)
print("compile 1, 1:", test_fns(f1, compile_f1, (inp1, inp2)))
print("compile 1, 2:", test_fns(f1, compile_f1, (-inp1, inp2)))
print("~" * 10)

compile 1, 1: True
compile 1, 2: True
~~~~~~~~~~


TorchScript scripting can handle data-dependent control flow, but this
solution comes with its own set of problems. Namely, TorchScript
scripting can require major code changes and will raise errors when
unsupported Python is used.

In the example below, we forget TorchScript type annotations and we
receive a TorchScript error because the input type for argument `y`, an
`int`, does not match with the default argument type, `torch.Tensor`.


In [14]:
def f2(x, y):
    return x + y

inp1 = torch.randn(5, 5)
inp2 = 3

script_f2 = torch.jit.script(f2)
try:
    script_f2(inp1, inp2)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_2246949/895162831.py", line 9, in <module>
    script_f2(inp1, inp2)
RuntimeError: f2() Expected a value of type 'Tensor (inferred)' for argument 'y' but instead found type 'int'.
Inferred 'y' to be of type 'Tensor' because it was not annotated with an explicit type.
Position: 1
Value: 3
Declaration: f2(Tensor x, Tensor y) -> Tensor
Cast error details: Unable to cast 3 to Tensor


However, `torch.compile` is easily able to handle `f2`.


In [15]:
compile_f2 = torch.compile(f2)
print("compile 2:", test_fns(f2, compile_f2, (inp1, inp2)))
print("~" * 10)

compile 2: True
~~~~~~~~~~


Another case that `torch.compile` handles well compared to previous
compilers solutions is the usage of non-PyTorch functions.


In [17]:
import scipy
def f3(x):
    x = x * 2
    x = scipy.fft.dct(x.numpy())
    x = torch.from_numpy(x)
    x = x * 2
    return x

TorchScript tracing treats results from non-PyTorch function calls as
constants, and so our results can be silently wrong.


In [18]:
inp1 = torch.randn(5, 5)
inp2 = torch.randn(5, 5)
traced_f3 = torch.jit.trace(f3, (inp1,))
print("traced 3:", test_fns(f3, traced_f3, (inp2,)))

traced 3: False


/tmp/ipykernel_2246949/123398134.py:4: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = scipy.fft.dct(x.numpy())
/tmp/ipykernel_2246949/123398134.py:5: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  x = torch.from_numpy(x)


TorchScript scripting and FX tracing disallow non-PyTorch function
calls.


In [19]:
try:
    torch.jit.script(f3)
except:
    tb.print_exc()

try:
    torch.fx.symbolic_trace(f3)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_2246949/1991772511.py", line 2, in <module>
    torch.jit.script(f3)
  File "/data/ywang/anaconda3/envs/diffusion/lib/python3.10/site-packages/torch/jit/_script.py", line 1341, in script
    fn = torch._C._jit_script_compile(
  File "/data/ywang/anaconda3/envs/diffusion/lib/python3.10/site-packages/torch/_jit_internal.py", line 1198, in _try_get_dispatched_fn
    return boolean_dispatched.get(fn)
  File "/data/ywang/anaconda3/envs/diffusion/lib/python3.10/weakref.py", line 453, in get
    return self.data.get(ref(key),default)
TypeError: cannot create weak reference to 'uarray._Function' object
Traceback (most recent call last):
  File "/tmp/ipykernel_2246949/1991772511.py", line 7, in <module>
    torch.fx.symbolic_trace(f3)
  File "/data/ywang/anaconda3/envs/diffusion/lib/python3.10/site-packages/torch/fx/_symbolic_trace.py", line 1109, in symbolic_trace
    graph = tracer.trace(root, concrete_args)
  File "/data/ywang/anacond

In comparison, `torch.compile` is easily able to handle the non-PyTorch
function call.


In [20]:
compile_f3 = torch.compile(f3)
print("compile 3:", test_fns(f3, compile_f3, (inp2,)))

compile 3: True


TorchDynamo and FX Graphs
=========================

One important component of `torch.compile` is TorchDynamo. TorchDynamo
is responsible for JIT compiling arbitrary Python code into [FX
graphs](https://pytorch.org/docs/stable/fx.html#torch.fx.Graph), which
can then be further optimized. TorchDynamo extracts FX graphs by
analyzing Python bytecode during runtime and detecting calls to PyTorch
operations.

Normally, TorchInductor, another component of `torch.compile`, further
compiles the FX graphs into optimized kernels, but TorchDynamo allows
for different backends to be used. In order to inspect the FX graphs
that TorchDynamo outputs, let us create a custom backend that outputs
the FX graph and simply returns the graph\'s unoptimized forward method.


In [21]:
from typing import List
def custom_backend(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print("custom backend called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward

# Reset since we are using a different backend.
torch._dynamo.reset()

opt_model = torch.compile(init_model(), backend=custom_backend)
opt_model(generate_data(16)[0])

custom backend called with FX graph:
`print_tabular` relies on the library `tabulate`, which could not be found on this machine. Run `pip install tabulate` to install the library.


BackendCompilerFailed: custom_backend raised UnboundLocalError: local variable 'tabulate' referenced before assignment

Set torch._dynamo.config.verbose=True for more information


You can suppress this exception and fall back to eager by setting:
    torch._dynamo.config.suppress_errors = True


Using our custom backend, we can now see how TorchDynamo is able to
handle data-dependent control flow. Consider the function below, where
the line `if b.sum() < 0` is the source of data-dependent control flow.


In [22]:
def bar(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b * -1
    return x * b

opt_bar = torch.compile(bar, backend=custom_backend)
inp1 = torch.randn(10)
inp2 = torch.randn(10)
opt_bar(inp1, inp2)
opt_bar(inp1, -inp2)

custom backend called with FX graph:
`print_tabular` relies on the library `tabulate`, which could not be found on this machine. Run `pip install tabulate` to install the library.


BackendCompilerFailed: custom_backend raised UnboundLocalError: local variable 'tabulate' referenced before assignment

Set torch._dynamo.config.verbose=True for more information


You can suppress this exception and fall back to eager by setting:
    torch._dynamo.config.suppress_errors = True


The output reveals that TorchDynamo extracted 3 different FX graphs
corresponding the following code (order may differ from the output
above):

1.  `x = a / (torch.abs(a) + 1)`
2.  `b = b * -1; return x * b`
3.  `return x * b`

When TorchDynamo encounters unsupported Python features, such as
data-dependent control flow, it breaks the computation graph, lets the
default Python interpreter handle the unsupported code, then resumes
capturing the graph.

Let\'s investigate by example how TorchDynamo would step through `bar`.
If `b.sum() < 0`, then TorchDynamo would run graph 1, let Python
determine the result of the conditional, then run graph 2. On the other
hand, if `not b.sum() < 0`, then TorchDynamo would run graph 1, let
Python determine the result of the conditional, then run graph 3.

This highlights a major difference between TorchDynamo and previous
PyTorch compiler solutions. When encountering unsupported Python
features, previous solutions either raise an error or silently fail.
TorchDynamo, on the other hand, will break the computation graph.

We can see where TorchDynamo breaks the graph by using
`torch._dynamo.explain`:


In [ ]:
# Reset since we are using a different backend.
torch._dynamo.reset()
explain_output = torch._dynamo.explain(bar)(torch.randn(10), torch.randn(10))
print(explain_output)

In order to maximize speedup, graph breaks should be limited. We can
force TorchDynamo to raise an error upon the first graph break
encountered by using `fullgraph=True`:


In [ ]:
opt_bar = torch.compile(bar, fullgraph=True)
try:
    opt_bar(torch.randn(10), torch.randn(10))
except:
    tb.print_exc()

And below, we demonstrate that TorchDynamo does not break the graph on
the model we used above for demonstrating speedups.


In [ ]:
opt_model = torch.compile(init_model(), fullgraph=True)
print(opt_model(generate_data(16)[0]))

We can use `torch.export` (from PyTorch 2.1+) to extract a single,
exportable FX graph from the input PyTorch program. The exported graph
is intended to be run on different (i.e. Python-less) environments. One
important restriction is that the `torch.export` does not support graph
breaks. Please check [this
tutorial](https://pytorch.org/tutorials/intermediate/torch_export_tutorial.html)
for more details on `torch.export`.


Conclusion
==========

In this tutorial, we introduced `torch.compile` by covering basic usage,
demonstrating speedups over eager mode, comparing to previous PyTorch
compiler solutions, and briefly investigating TorchDynamo and its
interactions with FX graphs. We hope that you will give `torch.compile`
a try!
